In [ ]:
import requests
import zipfile
import pandas as pd

**Задание 1**

Скачайте с сайта grouplens.org...movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

In [ ]:
 # download dataset.zip
ml_latest_small_url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
dataset_zip_path = '/tmp/ml-latest-small.zip'
resp = requests.get(ml_latest_small_url)
with open(file=dataset_zip_path, mode='wb') as dataset_file:
    dataset_file.write(resp.content)
    dataset_file.flush()

In [ ]:
# load CSV to dataframes
with zipfile.ZipFile(file=dataset_zip_path, compression=zipfile.ZIP_DEFLATED) as zip:

    ratings_zip_path = max(zip.namelist(), key=lambda rat_name: 'ratings.csv' in rat_name)
    print(ratings_zip_path)
    with zip.open(ratings_zip_path, 'r') as ratings_csv:
        ratings_df = pd.read_csv(filepath_or_buffer=ratings_csv, header=0)

    movies_zip_path = max(zip.namelist(), key=lambda mov_name: 'movies.csv' in mov_name)
    print(movies_zip_path)
    with zip.open(movies_zip_path, 'r') as movies_csv:
        movies_df = pd.read_csv(filepath_or_buffer=movies_csv, header=0, index_col='movieId')

ml-latest-small/ratings.csv
ml-latest-small/movies.csv


In [ ]:
the_top_movies = ratings_df[ratings_df['rating'] == 5.0]['movieId'].mode().squeeze()
movies_df.iloc[the_top_movies] # I don't

title     I Love Trouble (1994)
genres            Action|Comedy
Name: 360, dtype: object

**Задание 2**

По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [ ]:
# download a ZIP file with the dataset
power_csv_url = 'https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/15255/10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip'
resp = requests.get(power_csv_url)
homework_zip_path = '/tmp/pandas_10_homework.zip'
with open(homework_zip_path, 'wb') as homework_file:
    homework_file.write(resp.content)
    homework_file.flush()

In [ ]:
# load csv into DataFrame
with zipfile.ZipFile(homework_zip_path, 'r', zipfile.ZIP_DEFLATED) as homework_zip:
    homework_zip.printdir()

    power_path = max(homework_zip.namelist(), key=lambda entry_name: 'power.csv' in entry_name)
    with homework_zip.open(power_path, 'r') as power_csv:
        power_df = pd.read_csv(power_csv, header=0)

File Name                                             Modified             Size
power.csv                                      2018-06-06 18:02:48     29050479
transactions.csv                               2018-06-06 17:28:54         1989


In [ ]:
filter_func = lambda row: (row['country'] in ('Latvia', 'Lithuania', 'Estonia') and
                           row['category'] in (4, 12, 21) and
                           row['quantity'] > 0.0)
power_df[power_df.apply(func=filter_func, axis=1)]['quantity'].sum()

970728.0

**Задание 3**

Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.

Примеры страниц (необязательно брать именно эти):

https://fortrader.org/quotes

www.finanz.ru...om-vremeni

In [ ]:
import re

In [ ]:
# constants
bonds_9pct_active_url = 'https://old.rusbonds.ru/srch_simple.asp?go=1&status=T&bstav=8&estav=13&ex=0&s=1&d=1&p=' # append a page number
data_magic_no = 18
pages_magic_no = 19

In [ ]:
# get page count
tmp_bonds_df_lst = pd.read_html(bonds_9pct_active_url+'1')
page_cnt = max(map(int, re.findall('(\d+)', tmp_bonds_df_lst[pages_magic_no].at[0, 1])))
print(f'page count is {page_cnt}')
tmp_bonds_df_lst = None # free mem

page count is 19


In [ ]:
bonds_data = pd.DataFrame()
for page_num in range(1, page_cnt + 1):
    tmp_bonds_df_lst = pd.read_html(bonds_9pct_active_url+str(page_num))
    bonds_data = bonds_data.append(tmp_bonds_df_lst[data_magic_no])
    print(f'page #{page_num} was scraped')
tmp_bonds_df_lst = None # free mem

page #1 was scraped
page #2 was scraped
page #3 was scraped
page #4 was scraped
page #5 was scraped
page #6 was scraped
page #7 was scraped
page #8 was scraped
page #9 was scraped
page #10 was scraped
page #11 was scraped
page #12 was scraped
page #13 was scraped
page #14 was scraped
page #15 was scraped
page #16 was scraped
page #17 was scraped
page #18 was scraped
page #19 was scraped


In [ ]:
len(bonds_data)

951

In [ ]:
bonds_data.loc[0].head(4)

,Секторрынка,"Облигация, выпуск",Состояниевыпуска,Номеррегистрации,Началоразмещения,Датапогашения,Номинал,Объемэмиссии,Рейтинг
0,ЕвроКорп,Alliance Oil-3-2020-евро,В обращении,XS0925043100,24.04.2013,04.05.2023,1 000 USD,500 000 000,Есть
0,Корп,Автобан-Финанс-КО-01,В обращении,4CDE-01-82416-H,20.09.2019,16.09.2022,5 000 000 000 RUB,5 000 000 000,Есть
0,Корп,Банк ВТБ-Т2-3,В обращении,40901000B,03.12.2019,21.05.2030,10 000 000 RUB,20 000 000 000,Нет
0,Корп,ВсеИнструменты.ру-1-боб,В обращении,4B02-01-00410-R,13.12.2018,09.12.2021,1 000 RUB,700 000 000,Нет


In [ ]:
bonds_data.head(9)

,Секторрынка,"Облигация, выпуск",Состояниевыпуска,Номеррегистрации,Началоразмещения,Датапогашения,Номинал,Объемэмиссии,Рейтинг
0,ЕвроКорп,Alliance Oil-3-2020-евро,В обращении,XS0925043100,24.04.2013,04.05.2023,1 000 USD,500 000 000,Есть
1,ЕвроКорп,Barclays Bank-2027-евро,В обращении,XS0833618910,21.11.2012,03.02.2027,120 000 000 RUB,120 000 000,Нет
2,ЕвроКорп,BCS Structured-10-2025-ев,В обращении,XS2143253495,24.03.2020,30.06.2025,100 000 RUB,600 000 000,Нет
3,ЕвроКорп,BCS Structured-12-2023-ев,В обращении,XS1917700723,29.11.2018,19.11.2023,1 250 USD,10 000 000,Нет
4,ЕвроКорп,BCS Structured-12-2026-ев,В обращении,XS2369557371,04.08.2021,08.12.2026,1 250 USD,10 000 000,Нет
5,ЕвроКорп,BCS Structured-13-2023-ев,В обращении,XS1923604984,17.12.2018,11.12.2023,1 250 USD,10 000 000,Нет
6,ЕвроКорп,BCS Structured-15-2023-ев,В обращении,XS1943597994,01.02.2019,20.12.2023,100 000 RUB,500 000 000,Нет
7,ЕвроКорп,BCS Structured-16-2024-ев,В обращении,XS2028880115,17.07.2019,30.06.2024,100 000 RUB,600 000 000,Нет
8,ЕвроКорп,BCS Structured-20-2023-ев,В обращении,XS2104313221,17.01.2020,17.05.2023,1 250 USD,2 000 000,Нет


In [ ]:
bonds_data.tail(9)

,Секторрынка,"Облигация, выпуск",Состояниевыпуска,Номеррегистрации,Началоразмещения,Датапогашения,Номинал,Объемэмиссии,Рейтинг
33,Корп,ЮниМетрикс-01-об,В обращении,4-01-00402-R,09.09.2019,24.08.2022,1 000 RUB,400 000 000,Нет
34,Корп,ЮниСервис Капитал-02К-об,В обращении,4-02-00534-R,04.03.2021,21.01.2029,10 000 RUB,90 000 000,Нет
35,Муни,ЯНАО-35002-об,В обращении,RU35002YML0,07.12.2016,06.12.2023,1 000 RUB,20 000 000 000,Есть
36,Муни,ЯНАО-35003-об,В обращении,RU35003YML0,18.10.2017,18.10.2022,1 000 RUB,1 000 000 000,Есть
37,Муни,Ярославская Обл-35014-об,В обращении,RU35014YRS0,27.05.2016,19.05.2023,1 000 RUB,4 500 000 000,Есть
38,Муни,Ярославская Обл-35015-об,В обращении,RU35015YRS0,01.06.2017,20.04.2027,1 000 RUB,10 000 000 000,Есть
39,Корп,ЯТЭК-001Р-01,В обращении,4B02-01-20510-F-001P,03.11.2020,31.10.2023,1 000 RUB,4 000 000 000,Есть
40,Корп,ЯТЭК-001Р-02,В обращении,4B02-02-20510-F-001P,12.04.2021,07.04.2025,1 000 RUB,5 000 000 000,Есть
41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
